In [1]:
import pandas as pd
import os
from pandas import DataFrame

In [2]:
def normalized_values(y,dfmax, dfmin):
    a = (y- dfmin) / (dfmax - dfmin)
    return(a)

def apply_norm(normdf, unnormdf, col, dfmax, dfmin, dfmean, dfstd, norm_type):
    normdf[col] = unnormdf.apply(lambda x: normalized_values(x[col], dfmax, dfmin, dfmean, dfstd, norm_type), axis=1)

def dataset_sanity_check(df):
    for c in [cl for cl in df.columns if 'bin' not in cl]:
        print('column %s - max: %s, min : %s, mean: %s, std: %s'%(c, df[c].max(), df[c].min(), df[c].mean(), df[c].std()))

def normalize_dataset(df, d):
    X = DataFrame()
#    if aggrfile and os.path.exists(aggrfile):
#        with open(aggrfile) as aggrf:
#            aggrs = json.loads(aggrf.read())
    for c in df.columns:
        if c in d.keys():
            print(c)
            dfcfloat = df[c].astype('float64')
            #print("Normalize column:%s" % c)
            dfmax = d[c]['max'] if 'max' in d[c] else None
            dfmin = d[c]['min'] if 'min' in d[c] else None
            #print(d[c]['min'],'-',d[c]['max'])
            #dfmean = d[c]['mean'] if 'mean' in aggrs[c] else None
            #dfstd = d[c]['std'] if 'std' in aggrs[c] else None
            X[c] = dfcfloat.apply(lambda x: normalized_values(x, dfmax, dfmin))#, axis=1)
            #dataset_sanity_check(X[[c]])
        else:
            X[c] = df[c]
    return(X)

In [3]:
d = {'aspect': {'max': 359.99800000000016, 'min': -1.0},
 'curvature': {'max': 0.73661599999999994, 'min': -0.76867799999999997},
 'dem': {'max': 2806.1399999999999, 'min': -6.04},
 'dom_vel': {'max': 19.337629295696285, 'min': 0.49528943306704032},
 'evi': {'max': 0.99250000000000005, 'min': -0.19869999999999999},
 'lst_day': {'max': 342.77999999999997, 'min': 251.78},
 'lst_night': {'max': 322.45999999999998, 'min': 254.47999572753903},
 'max_dew_temp': {'max': 300.5537109375, 'min': 266.383056640625},
 'max_temp': {'max': 317.44775390625, 'min': 273.447509765625},
 'mean_dew_temp': {'max': 298.55347696940106, 'min': 263.98918660481769},
 'mean_temp': {'max': 309.74698893229169, 'min': 270.31141153971362},
 'min_dew_temp': {'max': 297.611083984375, 'min': 258.5595703125},
 'min_temp': {'max': 303.00634765625, 'min': 262.879150390625},
 'ndvi_new': {'max': 0.99939999999999996, 'min': -0.20000000000000001},
 'rain_7days': {'max': 3.5451469016037436, 'min': 6.8902961061212417e-06},
 'res_max': {'max': 19.337629295696285, 'min': 2.7861490154313331},
 'slope': {'max': 45.0045, 'min': 0.0},
    'x':{'max':28.04987944036008,'min':19.375364776587237},
    'y':{'max':41.74353364305825,'min':34.80164167588476},
    'frequency':{'max':3,'min':0},
    'f81':{'max':0.32845602804450552,'min':0}}

In [6]:
dict = {}
for folder, subfolders, files in os.walk('/users/pa21/sgirtsou/production', topdown=True):
    for file in files:
        if file.endswith('dummies.csv') and 'balanced' not in file:
            if file.split('_')[1] in dict:
                dict[file.split('_')[1]].append(os.path.join(os.path.abspath(folder), file))
            else:
                dict[file.split('_')[1]]=[]
                dict[file.split('_')[1]].append(os.path.join(os.path.abspath(folder), file))

In [7]:
dict

{'2020': ['/users/pa21/sgirtsou/production/2020/05/may_2020_dummies.csv',
  '/users/pa21/sgirtsou/production/2020/08/august_2020_dummies.csv',
  '/users/pa21/sgirtsou/production/2020/09/september_2020_dummies.csv',
  '/users/pa21/sgirtsou/production/2020/07/july_2020_dummies.csv',
  '/users/pa21/sgirtsou/production/2020/06/june_2020_dummies.csv']}

In [6]:
dict = {'2019': ['/work2/pa21/sgirtsou/production/2019/05/may_2019_dummies.csv',
  '/work2/pa21/sgirtsou/production/2019/08/august_2019_dummies.csv',
  '/work2/pa21/sgirtsou/production/2019/09/september_2019_dummies.csv',
  '/work2/pa21/sgirtsou/production/2019/04/april_2019_dummies.csv',
  '/work2/pa21/sgirtsou/production/2019/07/july_2019_dummies.csv',
  '/work2/pa21/sgirtsou/production/2019/06/june_2019_dummies.csv']}

In [8]:
xy = pd.read_csv('/users/pa21/sgirtsou/transfered_files/data/centroids_id_x_y.csv')
freq = pd.read_csv('/users/pa21/sgirtsou/ffp/code/ml-fires-1/ML_fires_al/stats/frequencyfeature_density_40km_1970_2009.csv')

In [9]:
for key in sorted(dict.keys()):
    print('Year:'+key)
    for i, filename in enumerate(dict[key]):
        print('Reading ', filename)
        df = pd.read_csv(filename)
        df['wkd'] = pd.to_datetime(df['firedate'],format='%Y%m%d').dt.dayofweek
        df['month'] = pd.to_datetime(df['firedate'],format='%Y%m%d').dt.month
        print('Getting the dummies ......')
        df1 = df
        df1 = pd.get_dummies(df, columns=['wkd', 'month'])
        del df
        months = ['month_4','month_5','month_6','month_7','month_8','month_9']
        for m in months:
            if m not in df1.columns:
                print(m+ 'was not in columns of' + filename)
                df1[m] = 0
        df1 = df1.merge(freq,left_on='id',right_on='id')
        df1 = df1.rename(columns={'81':'f81'})
        df1 = df1.merge(xy,left_on='id',right_on='id')
        print('Starting normalization....')
        df1 = normalize_dataset(df1,d)
        print('Saving normalized dataset....')
        df1.to_csv(filename[:-12]+'_norm.csv')

Year:2020
Reading  /users/pa21/sgirtsou/production/2020/05/may_2020_dummies.csv
Getting the dummies ......
month_4was not in columns of/users/pa21/sgirtsou/production/2020/05/may_2020_dummies.csv
month_6was not in columns of/users/pa21/sgirtsou/production/2020/05/may_2020_dummies.csv
month_7was not in columns of/users/pa21/sgirtsou/production/2020/05/may_2020_dummies.csv
month_8was not in columns of/users/pa21/sgirtsou/production/2020/05/may_2020_dummies.csv
month_9was not in columns of/users/pa21/sgirtsou/production/2020/05/may_2020_dummies.csv
Starting normalization....
max_temp
min_temp
mean_temp
res_max
dom_vel
rain_7days
dem
slope
curvature
aspect
ndvi_new
evi
lst_day
lst_night
max_dew_temp
mean_dew_temp
min_dew_temp
frequency
f81
x
y
Saving normalized dataset....
Reading  /users/pa21/sgirtsou/production/2020/08/august_2020_dummies.csv
Getting the dummies ......
month_4was not in columns of/users/pa21/sgirtsou/production/2020/08/august_2020_dummies.csv
month_5was not in columns of

In [11]:
df1.columns

Index(['id', 'firedate', 'max_temp', 'min_temp', 'mean_temp', 'res_max',
       'dom_vel', 'rain_7days', 'dem', 'slope', 'curvature', 'aspect',
       'ndvi_new', 'evi', 'lst_day', 'lst_night', 'max_dew_temp',
       'mean_dew_temp', 'min_dew_temp', 'fire', 'dir_max_1', 'dir_max_2',
       'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6', 'dir_max_7',
       'dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3', 'dom_dir_4',
       'dom_dir_5', 'dom_dir_6', 'dom_dir_7', 'dom_dir_8', 'corine_111',
       'corine_112', 'corine_121', 'corine_122', 'corine_123', 'corine_124',
       'corine_131', 'corine_132', 'corine_133', 'corine_141', 'corine_142',
       'corine_211', 'corine_212', 'corine_213', 'corine_221', 'corine_222',
       'corine_223', 'corine_231', 'corine_241', 'corine_242', 'corine_243',
       'corine_244', 'corine_311', 'corine_312', 'corine_313', 'corine_321',
       'corine_322', 'corine_323', 'corine_324', 'corine_331', 'corine_332',
       'corine_333', 'corine_334', 'c

In [12]:
df1[['firedate','wkd_0','wkd_1', 'wkd_2', 'wkd_3', 'wkd_4', 'wkd_5', 'wkd_6']]

,firedate,wkd_0,wkd_1,wkd_2,wkd_3,wkd_4,wkd_5,wkd_6
0,20200622,1,0,0,0,0,0,0
1,20200612,0,0,0,0,1,0,0
2,20200624,0,0,1,0,0,0,0
3,20200605,0,0,0,0,1,0,0
4,20200613,0,0,0,0,0,1,0
5,20200630,0,1,0,0,0,0,0
6,20200604,0,0,0,1,0,0,0
7,20200628,0,0,0,0,0,0,1
8,20200626,0,0,0,0,1,0,0
9,20200627,0,0,0,0,0,1,0


In [10]:
(filename[:-12]+'_norm.csv')

'/work2/pa21/sgirtsou/production/2018/09/september_2018_norm.csv'

In [23]:
df1.to_csv('/users/pa21/sgirtsou/july_2018_norm.csv')

In [ ]:
df = df.merge(freq,left_on='id',right_on='id')

In [ ]:
xy.y.max()

In [44]:
df.rename(columns={'81':'f81'})

,Unnamed: 0,id,firedate,max_temp,min_temp,mean_temp,res_max,dom_vel,rain_7days,dem,...,corine_334,corine_411,corine_412,corine_421,corine_422,corine_511,corine_512,corine_521,frequency,f81
0,0,333237,20100412,287.027588,277.887939,282.941050,5.193217,2.189361,0.079943,84.3599,...,0,0,0,0,0,0,0,0.0,0,0.000457
1,1,335462,20100412,286.617432,277.641846,282.461721,5.193217,2.220158,0.091805,77.4268,...,0,0,0,0,0,0,0,0.0,0,0.000457
2,2,335463,20100412,287.027588,277.887939,282.941050,5.193217,2.189361,0.079943,90.7271,...,0,0,0,0,0,0,0,0.0,0,0.000457
3,3,335464,20100412,287.027588,277.887939,282.941050,5.193217,2.189361,0.079943,84.3599,...,0,0,0,0,0,0,0,0.0,0,0.000457
4,4,337688,20100412,286.617432,277.641846,282.461721,5.193217,2.220158,0.091805,100.3930,...,0,0,0,0,0,0,0,0.0,0,0.000457
5,5,337689,20100412,286.617432,277.641846,282.461721,5.193217,2.220158,0.091805,92.0619,...,0,0,0,0,0,0,0,0.0,0,0.000457
6,6,335474,20100412,287.027588,277.887939,282.941050,5.193217,2.189361,0.079943,50.9403,...,0,0,0,0,0,0,0,0.0,0,0.000457
7,7,335475,20100412,287.027588,277.887939,282.941050,5.193217,2.189361,0.079943,46.1096,...,0,0,0,0,0,0,0,0.0,0,0.000457
8,8,337702,20100412,287.027588,277.887939,282.941050,5.193217,2.189361,0.079943,46.9248,...,0,0,0,0,0,0,0,0.0,0,0.000457
9,9,337703,20100412,287.027588,277.887939,282.941050,5.193217,2.189361,0.079943,45.0867,...,0,0,0,0,0,0,0,0.0,0,0.000457


# DATASETS 15

In [6]:
d = {'x':{'max':28.04987944036008,'min':19.375364776587237},
    'y':{'max':41.74353364305825,'min':34.80164167588476},
    'frequency':{'max':3,'min':0},
    'f81':{'max':0.32845602804450552,'min':0}}

In [7]:
freq = pd.read_csv('/users/pa21/sgirtsou/ffp/code/ml-fires-1/ML_fires_al/stats/frequencyfeature_density_40km_1970_2009.csv')
xy = pd.read_csv('/users/pa21/sgirtsou/transfered_files/data/centroids_id_x_y.csv')

In [8]:
df = pd.read_csv('/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity/balanced_2013_new.csv')

In [9]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'id', 'firedate',
       'max_temp', 'min_temp', 'mean_temp', 'res_max', 'dom_vel', 'rain_7days',
       'dem', 'slope', 'curvature', 'aspect', 'ndvi_new', 'evi', 'lst_day',
       'lst_night', 'max_dew_temp', 'mean_dew_temp', 'min_dew_temp', 'fire',
       'dir_max_1', 'dir_max_2', 'dir_max_3', 'dir_max_4', 'dir_max_5',
       'dir_max_6', 'dir_max_7', 'dir_max_8', 'dom_dir_1', 'dom_dir_2',
       'dom_dir_3', 'dom_dir_4', 'dom_dir_5', 'dom_dir_6', 'dom_dir_7',
       'dom_dir_8', 'corine_111', 'corine_112', 'corine_121', 'corine_122',
       'corine_123', 'corine_124', 'corine_131', 'corine_132', 'corine_133',
       'corine_141', 'corine_142', 'corine_211', 'corine_212', 'corine_213',
       'corine_221', 'corine_222', 'corine_223', 'corine_231', 'corine_241',
       'corine_242', 'corine_243', 'corine_244', 'corine_311', 'corine_312',
       'corine_313', 'corine_321', 'corine_322', 'corine_323', 'corine_324',
       'corine_33

In [10]:
df = df.drop(['Unnamed: 0','wkd_0', 'wkd_1', 'wkd_2', 'wkd_3', 'wkd_4', 'wkd_5', 'wkd_6',
       'month_5', 'month_4', 'month_6', 'month_7', 'month_8', 'month_9'],axis=1)

In [11]:
df['firedate'] = df['firedate'].astype('int64')

In [12]:
df['wkd'] = pd.to_datetime(df['firedate'],format='%Y%m%d').dt.dayofweek
df['month'] = pd.to_datetime(df['firedate'],format='%Y%m%d').dt.month
df = pd.get_dummies(df, columns=['wkd', 'month'])
months = ['month_4','month_5','month_6','month_7','month_8','month_9']
for m in months:
    if m not in df.columns:
        print(m+ 'was not in columns')
        df[m] = 0
df = df.merge(freq,left_on='id',right_on='id')
df = df.rename(columns={'81':'f81'})
df = df.merge(xy,left_on='id',right_on='id')
print('Starting normalization....')
df = normalize_dataset(df,d)
print('Saving normalized dataset....')
#df.to_csv(filename[:-12]+'_norm.csv')

Starting normalization....
frequency
f81
x
y
Saving normalized dataset....


In [14]:
df.to_csv('/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity/balanced_2013_dummies.csv')

checks on datasets

In [ ]:
df = pd.read_csv('/work2/pa21/sgirtsou/production/2013/06/june_2013_dummies.csv',nrows=1)

In [ ]:
df.columns